# Credit_scoring

## 1. Open the data file and have a look at the general information.

In [2]:
# import all required dependencies
import pandas as pd

# import NLTK and stemmer for working with texts
import nltk
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
from collections import Counter

#read the data and assign it to the credit_data variable
credit_data = pd.read_csv('credit_scoring_eng.csv')

In [3]:
#get familiarize with datas by sample 5 rows of data
credit_data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8939,0,-242.058742,37,secondary education,1,unmarried,4,F,employee,0,20763.285,university education
18752,0,-1527.196421,30,some college,2,unmarried,4,F,employee,0,27985.838,purchase of the house for my family
1915,0,373836.542882,67,bachelor's degree,0,widow / widower,2,F,retiree,0,36940.326,property
20517,1,-2886.028645,36,secondary education,1,married,0,M,business,0,30050.729,purchase of the house
4500,1,-886.057351,44,bachelor's degree,0,married,0,F,employee,0,14630.538,buy residential real estate


> it is better experience to have sample rows from data rather than just head of data, therefore ```sample()``` method is applied.

> ```dob_years``` column name is not clear at a first glance, but in the description of data it was provided that it is an age of customers. If so, let's change the column name to ```customer_age```. Other column names seems clear!

In [4]:
#rename the column name
credit_data.rename(columns={'dob_years':'customer_age'}, inplace=True)

In [5]:
# get know about the shape of data and info about each column's datatype
print('Table has {} rows, {} columns  and {} observations in total.'
      .format(credit_data.shape[0], credit_data.shape[1], (credit_data.shape[0]*credit_data.shape[1])))
print()
print()
print('Datatypes of columns are as follow:')
print('----------')
credit_data.info()

Table has 21525 rows, 12 columns  and 258300 observations in total.


Datatypes of columns are as follow:
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   customer_age      21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### 1.1 Conclusion

The required Python library - ```Pandas``` is imported and the provided data is opened successfully. With the printing of the head of the table, it has been observed that all columns stated in the job description are present! But the name of the ```dob_years``` column changed to ```customer_age``` for the sake of understanding.

The data contains  ```12``` columns, where 5 of them contains ```int64``` and another 5 contains ```object``` as well as 2 contains ```float64``` data types. There are ```21525``` rows in data, which make ```238300``` observations in total.

## 2. Data preprocessing

### 2.1 Processing missing values

In [6]:
# check total number of missing observations in the data
credit_data.isnull().sum().sum()

4348

> It is ovserved that there are ```4348``` missing observations in whole table, which makes just 1,8% of total observations. Before doing anything with this missing data, let's check which columns have missing value and is there any pattern related to missing values!

In [7]:
# check percentage of missing values in each column
credit_data.isnull().sum()*100/len(credit_data)

children             0.000000
days_employed       10.099884
customer_age         0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

> Only ```days_employed```and ```total_income``` columns have missing values. Number of missing values in both of them are exactly equal. Seems like they are related!

In [8]:
#retrieve data with rows that contain missing value on any of the columns
# and randomly select 10 or more rows with the sample() method
credit_data[credit_data.isnull().any(axis=1)].sample(10)

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15210,0,NaN,28,secondary education,1,divorced,3,F,employee,0,NaN,cars
12820,2,NaN,30,secondary education,1,unmarried,4,F,employee,1,NaN,buy real estate
1630,2,NaN,42,secondary education,1,married,0,F,employee,0,NaN,buy residential real estate
4476,0,NaN,30,Secondary Education,1,civil partnership,1,F,employee,0,NaN,purchase of the house
12828,0,NaN,47,bachelor's degree,0,married,0,F,civil servant,0,NaN,profile education
18385,0,NaN,21,secondary education,1,civil partnership,1,F,employee,0,NaN,buying property for renting out
11562,0,NaN,55,secondary education,1,civil partnership,1,F,retiree,0,NaN,property
2768,1,NaN,48,Bachelor's Degree,0,unmarried,4,F,civil servant,0,NaN,housing renovation
8294,0,NaN,28,bachelor's degree,0,married,0,M,business,0,NaN,building a property
16588,2,NaN,34,secondary education,1,married,0,M,business,0,NaN,education


In [9]:
# check 'income_type' of rows with missing values 
credit_data[credit_data.isnull().any(axis=1)]['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

> It can be seen that when the value of ```days_employed``` is missing, the value of ```total_income``` is also missing at the same rows. At first glance, it gives an impression that if the person is not employed, then he/she will not have an income and therefore both of them are simultaneously empty. But, when we check the ```income_type``` column, it is stated that most of these people are employee or so, who should have an income. That means either they were left intentionally blank or they are due to a technical error.

> Around 10% of rows in ```days_employed``` and ```total_income``` columns are missing. We can not throw them away. Let's try to fill them with the help of other columns.

In [10]:
# group observations according to income_type and gender, 
#hen retrieve mean and median of total_income
credit_data.groupby(['income_type','gender']).agg(
    {'total_income': ['mean', 'median']})

total_income            
                                            mean      median
income_type                 gender                          
business                    F       29475.489877  25731.3245
                            M       37284.334474  31491.0080
                            XNA     32624.825000  32624.8250
civil servant               F       24908.849830  21917.1980
                            M       34036.170503  29754.3915
employee                    F       23818.105441  20898.4980
                            M       28956.563225  25945.7880
entrepreneur                F       79866.103000  79866.1030
                            M                NaN         NaN
paternity / maternity leave F        8612.661000   8612.6610
retiree                     F       21446.167378  18529.2465
                            M       24117.476626  20918.3620
student                     M       15712.260000  15712.2600
unemployed                  F       32435.602000  32435.6020
                            M        9593.119000   9593.1190

> It is obvious that incomes of people doing different jobs are not equal and another sad truth is that average income of males and females are also not equal.

> Another issue with the data is that ```gender``` column has a strange value of "XNA", let's fix it first.

In [11]:
# find how many different values in gender column
credit_data.gender.value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

> only one row has value of 'XNA' for ```gender``` column. Most probably this is typos. The good thing is that its median value is very close to males' median. So we can assume it as a male and change its value to 'M'

In [12]:
# change value of XNA to M
credit_data.loc[credit_data.gender == 'XNA', 'gender'] = 'M'

In [13]:
# check if the issue was fixed
credit_data.gender.value_counts()

F    14236
M     7289
Name: gender, dtype: int64

In [14]:
# after fixing XNA, group observations according to income_type and gender, 
#hen retrieve mean and median of total_income
credit_data.groupby(['income_type','gender']).agg(
    {'total_income': ['mean', 'median']})

total_income            
                                            mean      median
income_type                 gender                          
business                    F       29475.489877  25731.3245
                            M       37281.604826  31498.3750
civil servant               F       24908.849830  21917.1980
                            M       34036.170503  29754.3915
employee                    F       23818.105441  20898.4980
                            M       28956.563225  25945.7880
entrepreneur                F       79866.103000  79866.1030
                            M                NaN         NaN
paternity / maternity leave F        8612.661000   8612.6610
retiree                     F       21446.167378  18529.2465
                            M       24117.476626  20918.3620
student                     M       15712.260000  15712.2600
unemployed                  F       32435.602000  32435.6020
                            M        9593.119000   9593.1190

> when we compare mean and median values, it can be seen that mostly there a big difference. That is due to the skewness of total income. Therefore, let's use median values of ```total_income``` grouped by ```income_type``` and ```gender``` columns in oder to fill missing values. 

In [15]:
# fill missing values in total_income according to the condition of income_type
# and gender columns
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='F')&
                 (credit_data.income_type == 'retiree')),'total_income']=218529.2465
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='M')&
                 (credit_data.income_type == 'retiree')),'total_income']=20918.3620
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='F')&
                 (credit_data.income_type == 'employee')),'total_income']=20898.4980
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='M')&
                 (credit_data.income_type == 'employee')),'total_income']=25945.7880
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='F')&
                 (credit_data.income_type == 'civil servant')),'total_income']=21917.1980
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='M')&
                 (credit_data.income_type == 'civil servant')),'total_income']=29754.3915
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='F')&
                 (credit_data.income_type == 'business')),'total_income']=25731.3245
credit_data.loc[((credit_data.total_income.isnull())&(credit_data.gender=='M')&
                 (credit_data.income_type == 'business')),'total_income']=31498.3750
credit_data.loc[((credit_data.total_income.isnull())&
                 (credit_data.income_type == 'entrepreneur')),'total_income']=79866.1030

In [16]:
#check if all missings were fixed in total_income column
credit_data.total_income.isnull().sum()

0

> Missing values of ```total_income``` was fixed. Now we need to fill missing values of ```days_employed``` column. We can apply exactly the same method as used to fill ```total_income```. However ```days_employed``` columns values are weird, there are negative values and very large positive numbers like 395302. Let's try to understand what are they, why they look like that?

In [17]:
credit_data[credit_data.days_employed > 0]['income_type'].value_counts()

retiree       3443
unemployed       2
Name: income_type, dtype: int64

In [18]:
credit_data.loc[credit_data.days_employed <= 0]['income_type'].value_counts()

employee                       10014
business                        4577
civil servant                   1312
student                            1
entrepreneur                       1
paternity / maternity leave        1
Name: income_type, dtype: int64

> When we analyze the ```days_employed``` column together with ```income_type``` column together. We have observed that, ```days_emploayed``` contains large positive numbers only in case when ```income_type``` is either retiree or unemployed! Otherwise, i.e. for any other ```income_type``` they are negative numbers. From here, it can be concluded that "-" sign was used to differentiate between employed vs unemployed.

> Moreover, when we devide positive numbers to 365 in order to get years, we got unrealistic years as employed.

**At this point it is not clear what the numbers in ```days_employed``` actually mean.**

> However, these column is not important currently. Let's simply fill the missing values as we did ```total_income``` column (but omitting ```gender``` column).

In [19]:
# get mean and median values of days_employed according to income_type
credit_data.groupby(['income_type']).agg({'days_employed': ['mean', 'median']})

days_employed               
                                      mean         median
income_type                                              
business                      -2111.524398   -1547.382223
civil servant                 -3399.896902   -2689.368353
employee                      -2326.499216   -1574.202821
entrepreneur                   -520.848083    -520.848083
paternity / maternity leave   -3296.759962   -3296.759962
retiree                      365003.491245  365213.306266
student                        -578.751554    -578.751554
unemployed                   366413.652744  366413.652744

> Here also, mean and median values differentiate a lot due to outlier. Therefore let's use median values to fill missing values.

In [20]:
#fill missing values in days_employed columns with the median values
credit_data.loc[((credit_data.days_employed.isnull())&
                 (credit_data.income_type == 'retiree')),'days_employed']=365213.306266
credit_data.loc[((credit_data.days_employed.isnull())&
                 (credit_data.income_type == 'employee')),'days_employed']=-1574.202821
credit_data.loc[((credit_data.days_employed.isnull())&
                 (credit_data.income_type == 'civil servant')),'days_employed']=-2689.368353
credit_data.loc[((credit_data.days_employed.isnull())&
                 (credit_data.income_type == 'business')),'days_employed']=-1547.382223
credit_data.loc[((credit_data.days_employed.isnull())&
                 (credit_data.income_type == 'entrepreneur')),'days_employed']=366413.652744

In [21]:
# check if all missing values of data was fixed
credit_data.isnull().sum().sum()

0

### 2.2 Conclusion

There were missing values in ```days_employed``` and ```total_income``` columns. In total 10% of values were missing in each of the columns. Missing values were occurring in the same rows for the two columns. 
The total income of different job types and gender was not equal. Moreover, there was an outlier in the data which resulted in different mean and median values. Hence, missing values of the ```total_income``` column are filled according to the median values of total income depending on ```income_type``` and ``` gender``` columns.
Missing values of ```days_employed``` column was filled with median values days employed depending on ```income_type```. However, it was observed that **retiree** and **unemployed** customers assigned with relatively large positive numbers, in contrast, **employee**, **business** etc customers assigned with negative numbers. Actually, negative numbers can be converted to years by taking absolute value and dividing the value with 365. However, a positive number resulted in very large unrealistic years. So at this point, no further work done on the ```days_employed``` column.

### 3. Data type replacement

In [22]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   customer_age      21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [23]:
# get insight about the descriptive statistics of columns which stores numerical values!
credit_data.describe()

,children,days_employed,customer_age,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000
mean,0.538908,63567.543970,43.293380,0.817236,0.972544,0.080883,29706.305172
std,1.381587,141164.838543,12.574584,0.548138,1.420324,0.272661,28956.024285
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2570.047544,33.000000,1.000000,0.000000,0.000000,17247.708000
50%,0.000000,-1355.683356,42.000000,1.000000,0.000000,0.000000,23520.274000
75%,1.000000,-316.061820,53.000000,1.000000,1.000000,0.000000,32083.737000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


> From the descriptive statistics and information about columns, we can see that max/min numbers for ```children, customer_age, education_id, family_status_id, debt``` columns are between -128 and 127. That means instead of using ```int64``` data type, we can use ```int8``` which will save memory (currently 2.0+ MB).

> ```days_employed``` and ```total_income``` columns also have a floating point data type. For the sake of simplicity, lets get rid of the points or simply convert them into integer numbers.

> Also we have an issue with ```children``` columns, which says mininum value is -1 and max is 20. These values are unrealistic. Seems like a typos. 

>Let's fix these issues one by one. 

In [24]:
#loop thourgh give columns and change their data type into int8
for col in ['children', 'customer_age', 'education_id', 'family_status_id', 'debt']:
    credit_data[col] = credit_data[col].astype('int8')

# convert into int64 from float64 for two columns
credit_data.days_employed = credit_data.days_employed.astype('int64')
credit_data.total_income = credit_data.total_income.astype('int64')

In [25]:
# check number of different value in children column
credit_data.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [26]:
#change values of the children column with values of -1 and 20 to 1 and 2 respectively.
credit_data.loc[(credit_data['children'] == -1), 'children'] = 1
credit_data.loc[(credit_data['children'] == 20), 'children'] = 2

#check if it worked
credit_data.children.value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [27]:
#check if all worked
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int8  
 1   days_employed     21525 non-null  int64 
 2   customer_age      21525 non-null  int8  
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int8  
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int8  
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int8  
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(2), int8(5), object(5)
memory usage: 1.3+ MB


> Now memory decreased from 2.0+MB to 1.3+MB and data type of columns changed. By the way, it is easy to read the ```days_employed``` and ```total_income``` columns.

### 3.1 Conclusion

Data type of ```'children', 'customer_age', 'education_id', 'family_status_id', 'debt'``` columns changed from ```int64``` to ```int8``` to save memory. Also, it was hard to read values of ```days_employed and total_income``` columns due to the floating points. These columns' data type also changed to ```int64``` from ```float64```. There are artefacts observed in ```childrens``` column, such as -1 or 20 children. This was assumed as typing mistakes and these values changed to 1 and 2, respectively.

### 4. Processing duplicates

In [28]:
#find number of duplicated rows in data
credit_data.duplicated().sum()

54

In [29]:
# print unique values in categorical columns, to check if there is low/uppercase issue
print(credit_data.education.unique())
print()
print(credit_data.family_status.unique())
print()
print(credit_data.income_type.unique())
print()
print(credit_data.purpose.unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'


> It is obvious that education column has a lowercase and uppercase words. We should convert them all into lowercase

> Also, in ```purpose``` columns, they purposes are described with different words. For that we will use NLTK library and make a new column which contains purpose categories.

In [30]:
# convert all values in education column into lower case.
credit_data.education = credit_data.education.str.lower()
credit_data.education.unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [31]:
# collect purposes in all rows, and then make a big list containing all words
# go through all words and get their stems. Sort stems according to their frequency with Counter
words_list =[]
for sentence in credit_data.purpose:
    words = nltk.word_tokenize(sentence)
    for word in words:
        stemmed = stemmer.stem(word)
        words_list.append(stemmed)
Counter(words_list).most_common()

[('a', 5130),
 ('real', 4478),
 ('estat', 4478),
 ('car', 4315),
 ('buy', 4004),
 ('hous', 3820),
 ('educ', 3526),
 ('purchas', 3314),
 ('to', 3081),
 ('of', 2998),
 ('transact', 2610),
 ('properti', 2542),
 ('my', 2396),
 ('wed', 2348),
 ('own', 2240),
 ('have', 1551),
 ('get', 1316),
 ('commerci', 1315),
 ('for', 1294),
 ('the', 1288),
 ('with', 1281),
 ('build', 1246),
 ('second-hand', 968),
 ('univers', 949),
 ('supplementari', 909),
 ('ceremoni', 797),
 ('rent', 653),
 ('out', 653),
 ('famili', 641),
 ('construct', 635),
 ('renov', 612),
 ('residenti', 607),
 ('go', 496),
 ('an', 443),
 ('profil', 436),
 ('higher', 426),
 ('becom', 412)]

In [32]:
# construct a function, that goes thorough pupose column and 
#cagetorize them according to the stems
def purpose_shorter(purpose):
    """takes purpose, makes lsit of words in it and then obtains stems of those words, 
    while producing list of stems. Later it checks each purpose if it contains 
    specific stem, then assing categories: house, car, education and wedding"""
    stems =[]
    words = nltk.word_tokenize(purpose)
    for word in words:
        stemmed = stemmer.stem(word)
        stems.append(stemmed)
    if ('hous' in stems) or ('properti' in stems) or ('estat' in stems):
        return 'house'
    if 'car' in stems:
        return 'car'
    if ('educ'in stems) or ('univers' in stems):
        return 'education'
    if 'wed' in stems:
        return 'wedding'
    else:
        return float('NaN')

        
# check if the function is working properly
print(purpose_shorter(credit_data.purpose[0]))

house


In [33]:
#apply the function the data and make a new column
credit_data['purpose_category'] = credit_data.purpose.apply(purpose_shorter)

#check new data
credit_data.sample(5)

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
8767,0,-1547,28,secondary education,1,unmarried,4,M,business,0,31498,housing transactions,house
5580,0,-4978,34,secondary education,1,unmarried,4,F,employee,0,16775,building a real estate,house
9389,0,-739,49,secondary education,1,married,0,M,employee,1,17416,construction of own property,house
1101,0,398750,51,secondary education,1,married,0,F,retiree,0,11989,housing,house
8745,1,375498,53,secondary education,1,divorced,3,F,retiree,0,35573,purchase of my own house,house


> Now we can drop intial ```purpose``` column, in that way we can better see the duplicates in our data. Because, maybe purpose of the applicant is recorded with different words, such as "house","real estate", "buying house" or "housing" etc.

In [34]:
#drop purpose column
credit_data.drop('purpose', inplace=True, axis=1)

In [35]:
# check new state of data
credit_data.head(3)

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_category
0,1,-8437,42,bachelor's degree,0,married,0,F,employee,0,40620,house
1,1,-4024,36,secondary education,1,married,0,F,employee,0,17932,car
2,0,-5623,33,secondary education,1,married,0,M,employee,0,23341,house


In [36]:
credit_data.duplicated().sum()

405

In [37]:
# what percentage of rows duplicated
credit_data.duplicated().sum()*100/len(credit_data)

1.8815331010452963

In [38]:
credit_data.drop_duplicates(inplace=True)

### 4.1 Conclusion

In the beginning, there were only **54** duplicates. But after converting all strings into lowercase and organizing purpose into categories, we have **405** duplicated rows.
In our data, we do not have a unique identifier for each customer. That makes things a little bit confusing. But the good thing is we have 12 columns in total, which lowers the probability of different customers having the same indicators. Additionally, duplicated rows are less than 2% of total rows. Based on these we can drop all duplicated rows.

## 5. Categorizing Data

> According to the questions asked to us, we need to categorize our ```total_income``` column. Note that we have already categorized the ```purpose```` column in previous section.

In [39]:
# get an insight about the diestrubition of values in total_income column
credit_data.total_income.describe()

count     21120.000000
mean      28756.044176
std       25634.289197
min        3306.000000
25%       17089.000000
50%       23458.500000
75%       32050.750000
max      362496.000000
Name: total_income, dtype: float64

> Let's categorize ```total_income``` data using its quartiles. 1st quartile (25%) would be **low**, 2nd quartile (50%) - **medium**, 3rd quartile (75%) - **high** and anything else higher than 75% would be **very high** income categories.

In [40]:
#write a function that makes category according to the amount of total income
def income_categorizer(income):
    if income < 17089:
        return 'low'
    if income < 23458.5:
        return 'middle'
    if income < 32050.75:
        return 'high'
    else:
        return 'very high'

#apply the fucntion to the data
credit_data['income_level'] = credit_data['total_income'].apply(income_categorizer)

In [41]:
# check new data with income category
credit_data.head()

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_category,income_level
0,1,-8437,42,bachelor's degree,0,married,0,F,employee,0,40620,house,very high
1,1,-4024,36,secondary education,1,married,0,F,employee,0,17932,car,middle
2,0,-5623,33,secondary education,1,married,0,M,employee,0,23341,house,middle
3,3,-4124,32,secondary education,1,married,0,M,employee,0,42820,education,very high
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,wedding,high


### 5.1 Conclusion

Categorization of the purposes was performed in the previous task. So here we have categorized total_income each customer get according to if they are below or above the specific quartile of the data. Briefly:

1st quartile (25%) - **low**

2nd quartile (50%) - **medium**

3rd quartile (75%) - **high** 

higher than 75% - **very high**.

## 6. Answering the questions

- **Q1: Is there a relation between having kids and repaying a loan on time?**

In [42]:
# constuct a pivot table with counting customer number 
#who defauled or not according to their number of children
pivot_table_kid = credit_data.pivot_table(
    index='debt', columns='children', aggfunc={'debt':['count']})
pivot_table_kid

debt                                  
            count                                  
children        0       1       2      3     4    5
debt                                               
0         12768.0  4353.0  1913.0  302.0  36.0  9.0
1          1061.0   445.0   202.0   27.0   4.0  NaN

In [43]:
#calculate the percentage of customers who defaulted out of total customers
# and add this percantge as a third row of the pivot table
pivot_table_kid.loc['defaulted %'] = pivot_table_kid.loc[1]*100/(pivot_table_kid.loc[0] + pivot_table_kid.loc[1])
pivot_table_kid

debt                                                 
                    count                                                 
children                0            1            2           3     4    5
debt                                                                      
0            12768.000000  4353.000000  1913.000000  302.000000  36.0  9.0
1             1061.000000   445.000000   202.000000   27.000000   4.0  NaN
defaulted %      7.672283     9.274698     9.550827    8.206687  10.0  NaN

- **A1: Answer to the Q1**
Most of the customers don't have any children and they have the lowest percentage of customers who defaulted. It is hard to compare data in this way because the population of customers with a high number of kids (e.g. there are only 329 people with three kid, while 12829 people without the kid) are significantly low than people who do not have a kid. Consequently, we can say that defaulting is not strongly related to having a kid.

- **Q2: Is there a relation between marital status and repaying a loan on time?**

In [44]:
# make a pivot table as in previous task
pivot_marital = credit_data.pivot_table(index='debt', columns='family_status',  aggfunc={'debt':['count']})
pivot_marital.loc['defaulted %'] = pivot_marital.loc[1]*100/(pivot_marital.loc[0] + pivot_marital.loc[1])
pivot_marital

debt                                          \
                          count                                           
family_status civil partnership     divorced       married    unmarried   
debt                                                                      
0                   3736.000000  1108.000000  11147.000000  2510.000000   
1                    388.000000    85.000000    929.000000   274.000000   
defaulted %            9.408341     7.124895      7.692945     9.841954   

                               
                               
family_status widow / widower  
debt                           
0                  880.000000  
1                   63.000000  
defaulted %          6.680806

- **A2: Answer to the Q2**
Based on the percentages of customers who defaulted, we can say that people who are **unmarried** or **civil partnership** has a higher defaulting rate. Maybe, that kind of people feels less responsibility :). In contrast, **widow** or **divorced** people are more likely to pay their debts on time.

- **Q3: Is there a relation between income level and repaying a loan on time?**

In [45]:
# make a pivot table as in previous task
pivot_income = credit_data.pivot_table(index='debt', columns='income_level',  aggfunc={'debt':['count']})
pivot_income.loc['defaulted %'] = pivot_income.loc[1]*100/(pivot_income.loc[0] + pivot_income.loc[1])
pivot_income

debt                                      
                    count                                      
income_level         high         low       middle    very high
debt                                                           
0             4805.000000  4863.00000  4815.000000  4898.000000
1              475.000000   416.00000   466.000000   382.000000
defaulted %      8.996212     7.88028     8.824086     7.234848

- **A3: Answer to the Q3**
There is no clear correlation between income level and defaulting. However, based on the percentages, we can say that people with **very high** and **low** income are more responsible for their debts. Whereas **middle** and **high** income people more likely to default.

- **Q4: How do different loan purposes affect on-time repayment of the loan?**

In [46]:
# make a pivot table as in prevous task
pivot_purpose = credit_data.pivot_table(index='debt', columns='purpose_category',  aggfunc={'debt':['count']})
pivot_purpose.loc['defaulted %'] = pivot_purpose.loc[1]*100/(pivot_purpose.loc[0] + pivot_purpose.loc[1])
pivot_purpose

debt                                       
                        count                                       
purpose_category          car    education        house      wedding
debt                                                                
0                 3870.000000  3594.000000  9797.000000  2120.000000
1                  402.000000   370.000000   781.000000   186.000000
defaulted %          9.410112     9.334006     7.383248     8.065915

- **A4: Answer to the Q4**
People who have borrowed money for construction or buying a house are less likely to default compared to other purposes. While people who had a purpose of **car** or **education** has a higher risk of default.

## 7. Summary

The data about bank customer's debt repaying was successfully opened and analyzed. Missing values were detected, which might be intentionally left blank or technical error, and these missing values were filled using median values depending on values of other indicators. The number of duplicates increased when categorical columns are changed to lower case and the purpose column was changed to a shorter purpose category. Each question asked initially was answered depending on the data. However, there was not drastic relationship observed.